In [1]:
import pandas as pd
import numpy as np
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from sklearn.preprocessing import normalize
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering

In [2]:
df = pd.read_csv('frappe.csv', sep="\t")
df.head()

,user,item,cnt,daytime,weekday,isweekend,homework,cost,weather,country,city
0,0,0,1,morning,sunday,weekend,unknown,free,sunny,United States,0
1,1,1,7,afternoon,saturday,weekend,unknown,free,cloudy,Spain,0
2,2,2,6,evening,monday,workday,unknown,free,cloudy,Spain,369
3,3,3,1,sunset,thursday,workday,unknown,free,unknown,United States,1028
4,4,4,428,night,thursday,workday,home,free,sunny,Switzerland,147


In [3]:
df = df[df['country'].isin(list(df['country'].value_counts()[:10].index))]

In [4]:
meta_app = pd.read_csv('meta.csv', sep="\t")

In [5]:
df = df.merge(meta_app, on='item')

In [6]:
df = df[['user','item','category','cnt','daytime','weekday','isweekend','homework','cost','weather','country']]

In [7]:
df[df['cnt']==28752.000000]

,user,item,category,cnt,daytime,weekday,isweekend,homework,cost,weather,country
52465,70,336,Business,28752,afternoon,saturday,weekend,unknown,free,unknown,unknown


In [8]:
def transfer(x):
    return 1 + (5-1)/(28752-1) *(x - 1) 

In [9]:
df['cnt'] = df['cnt'].map(transfer)

In [10]:
context = pd.concat([df[['user','item','category','cnt']],pd.get_dummies(df.iloc[:,4:])],axis=1)

In [11]:
# context_test_user = context.copy()

In [15]:
context_test_user = context.copy()
user_context_correlation = dict()
cluster_set = list(set(context_test_user['category'].values))
from collections import defaultdict
for cluster in cluster_set:
    user_context_correlation.setdefault(cluster,defaultdict())
    test = context_test_user[context_test_user['category'] == cluster]
    for context_factor in test.columns[4:]:
        X=np.vstack([test['cnt'].values,test[context_factor].values])
        d2=np.corrcoef(X)[0][1]
        user_context_correlation[cluster][context_factor] = d2
        

/home/jinfeng/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/jinfeng/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [16]:
context_test_user

,user,item,category,cnt,daytime_afternoon,daytime_evening,daytime_morning,daytime_night,daytime_noon,daytime_sunrise,...,country_Australia,country_Canada,country_Germany,country_Israel,country_Mexico,country_Peru,country_Spain,country_United Kingdom,country_United States,country_unknown
0,0,0,Productivity,1.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,9,0,Productivity,1.000000,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,181,0,Productivity,1.003478,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,248,0,Productivity,1.015582,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,47,0,Productivity,1.002783,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75295,273,4074,Arcade & Action,1.000417,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
75296,183,4075,Entertainment,1.003478,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
75297,123,4076,Education,1.000557,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
75298,247,4079,Brain & Puzzle,1.000000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
context_user_correlation = dict()
for context_factor in context_test_user.columns[4:]:
    context_user_correlation.setdefault(context_factor,defaultdict())
    for cluster in cluster_set:
        test = context_test_user[context_test_user['user'] == cluster]
        X=np.vstack([test['cnt'].values,test[context_factor].values])
        d2=np.corrcoef(X)[0][1]
        context_user_correlation[context_factor][cluster] = d2
    
    

/home/jinfeng/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/home/jinfeng/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/jinfeng/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:162: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/jinfeng/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/jinfeng/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/home/jinfeng/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: invalid value encoun